In [15]:
import spike.spike_analysis.spike_collection as sc
import spike.spike_analysis.spike_recording as sr
import spike.spike_analysis.firing_rate_calculations as fr
import spike.spike_analysis.normalization as norm
import spike.spike_analysis.single_cell as single_cell
import spike.spike_analysis.spike_collection as collection
import spike.spike_analysis.zscoring as zscoring
import importlib
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import behavior.boris_extraction as boris
import matplotlib.pyplot as plt
import pickle
import re

In [16]:
pd.set_option('display.max_colwidth', 0)  # 0 means unlimited in newer pandas versions

# Show all rows
pd.set_option("display.max_rows", None)

# Show all columns
pd.set_option("display.max_columns", None)

# Don’t truncate column contents
pd.set_option("display.max_colwidth", None)

# Expand the display to the full width of the screen
pd.set_option("display.width", 0)


In [17]:
spike_collection_json_path = r'C:\Users\thoma\Code\ResearchCode\diff_fam_social_memory_ephys\spike_collection.json\spike_collection.json'

### Loading spike Collection

In [18]:
sp = sc.SpikeCollection.load_collection(spike_collection_json_path)

In [19]:
rec_events = sp.recordings[0].event_dict

# get unique event names from rec_events dictionary
event_names = list(rec_events.keys())
print("Unique event names:", event_names)

Unique event names: ['alone_rewarded', 'alone_rewarded_baseline', 'high_comp', 'high_comp_lose', 'high_comp_lose_baseline', 'high_comp_win', 'high_comp_win_baseline', 'lose', 'low_comp', 'low_comp_lose', 'low_comp_lose_baseline', 'low_comp_win', 'low_comp_win_baseline', 'overall_pretone', 'win']


### Loading Alternative Event Dict and Original (with frames)

### Alternative (Taylors)

In [20]:
# Load the alternative event dict
alternative_event_dict_path = r'C:\Users\thoma\Code\ResearchCode\diff_fam_social_memory_ephys\other_peoples_sutff\Thomas\reward_comp\testing_eventdict_creation\alternative_event_dict_20251003_082057.pkl'

with open(alternative_event_dict_path, 'rb') as f:
    alternative_event_dict = pickle.load(f)

print(alternative_event_dict.keys())

dict_keys(['20230612_101430_standard_comp_to_training_D1_subj_1-3', '20230612_101430_standard_comp_to_training_D1_subj_1-4', '20230612_112630_standard_comp_to_training_D1_subj_1-1', '20230612_112630_standard_comp_to_training_D1_subj_1-2', '20230613_105657_standard_comp_to_training_D2_subj_1-1', '20230613_105657_standard_comp_to_training_D2_subj_1-4', '20230614_114041_standard_comp_to_training_D3_subj_1-1', '20230614_114041_standard_comp_to_training_D3_subj_1-2', '20230616_111904_standard_comp_to_training_D4_subj_1-2', '20230616_111904_standard_comp_to_training_D4_subj_1-4', '20230617_115521_standard_comp_to_omission_D1_subj_1-1', '20230617_115521_standard_comp_to_omission_D1_subj_1-2', '20230618_100636_standard_comp_to_omission_D2_subj_1-1', '20230618_100636_standard_comp_to_omission_D2_subj_1-4', '20230619_115321_standard_comp_to_omission_D3_subj_1-2', '20230619_115321_standard_comp_to_omission_D3_subj_1-4', '20230620_114347_standard_comp_to_omission_D4_subj_1-1', '20230620_114347_sta

#### Originalish (Frames)

In [21]:
# Load the original event dict
original_event_dict_path = r'C:\Users\thoma\Code\ResearchCode\diff_fam_social_memory_ephys\other_peoples_sutff\Thomas\reward_comp\testing_eventdict_creation\original_event_dict_20251003_082057.pkl'

with open(original_event_dict_path, 'rb') as f:
    original_event_dict = pickle.load(f)

print(original_event_dict.keys())

dict_keys(['20230612_101430_standard_comp_to_training_D1_subj_1-4_t4b2L_box1_merged.rec', '20230612_101430_standard_comp_to_training_D1_subj_1-3_t3b3L_box2_merged.rec', '20230612_112630_standard_comp_to_training_D1_subj_1-2_t2b2L_box1_merged.rec', '20230612_112630_standard_comp_to_training_D1_subj_1-1_t1b3L_box2_merged.rec', '20230613_105657_standard_comp_to_training_D2_subj_1-1_t1b2L_box1_merged.rec', '20230613_105657_standard_comp_to_training_D2_subj_1-4_t4b3L_box2_merged.rec', '20230614_114041_standard_comp_to_training_D3_subj_1-1_t1b3L_box1_merged.rec', '20230614_114041_standard_comp_to_training_D3_subj_1-2_t2b2L_box2_merged.rec', '20230616_111904_standard_comp_to_training_D4_subj_1-4_t4b3L_box1_merged.rec', '20230616_111904_standard_comp_to_training_D4_subj_1-2_t2b2L_box2_merged.rec', '20230617_115521_standard_comp_to_omission_D1_subj_1-1_t1b3L_box1_merged.rec', '20230617_115521_standard_comp_to_omission_D1_subj_1-2_t2b2L_box2_merged.rec', '20230618_100636_standard_comp_to_omissio

In [22]:
test_rec = alternative_event_dict["20230612_101430_standard_comp_to_training_D1_subj_1-3"]

# Let's examine the structure of an event dictionary entry
print("Keys in test_rec:", list(test_rec.keys()))
print("\nSample event structure:")
sample_event_key = list(test_rec.keys())[0]
print(f"Event '{sample_event_key}':")
print(f"Type: {type(test_rec[sample_event_key])}")
print(f"Sample values: {test_rec[sample_event_key][:10] if hasattr(test_rec[sample_event_key], '__len__') else test_rec[sample_event_key]}")

Keys in test_rec: ['rewarded', 'omission', 'both_rewarded', 'tie', 'no_comp_win', 'no_comp_lose', 'competitive_win', 'competitive_lose']

Sample event structure:
Event 'rewarded':
Type: <class 'numpy.ndarray'>
Sample values: []


#### Looking at Timestamps max time value to compare to spikes

In [23]:
'''
# Find maximum values across all events in both dictionaries

def find_max_values_in_event_dict(event_dict, dict_name):
    """Find the maximum value across all events in all recordings in an event dictionary"""
    all_values = []
    
    
    for recording_name, recording_events in event_dict.items():        
        for event_name, event_values in recording_events.items():
            if len(event_values) > 0:  # Only process non-empty events
                max_val = np.max(event_values)
                all_values.extend(event_values)
            else:
                print(f"  {event_name}: empty")
    
    if all_values:
        overall_max = np.max(all_values)
        return overall_max, all_values
    else:
        print(f"\nNo values found in {dict_name}")
        return None, []

# Find max values in alternative event dict
alt_max, alt_values = find_max_values_in_event_dict(alternative_event_dict, "Alternative Event Dict")

# Find max values in original event dict  
orig_max, orig_values = find_max_values_in_event_dict(original_event_dict, "Original Event Dict")

# Compare the overall maximums
print(f"\n=== SUMMARY ===")
if alt_max is not None:
    print(f"Alternative Event Dict - Overall Max: {alt_max}")
if orig_max is not None:
    print(f"Original Event Dict - Overall Max: {orig_max}")

if alt_max is not None and orig_max is not None:
    print(f"Difference: {abs(alt_max - orig_max)}")

'''

'\n# Find maximum values across all events in both dictionaries\n\ndef find_max_values_in_event_dict(event_dict, dict_name):\n    """Find the maximum value across all events in all recordings in an event dictionary"""\n    all_values = []\n    \n    \n    for recording_name, recording_events in event_dict.items():        \n        for event_name, event_values in recording_events.items():\n            if len(event_values) > 0:  # Only process non-empty events\n                max_val = np.max(event_values)\n                all_values.extend(event_values)\n            else:\n                print(f"  {event_name}: empty")\n    \n    if all_values:\n        overall_max = np.max(all_values)\n        return overall_max, all_values\n    else:\n        print(f"\nNo values found in {dict_name}")\n        return None, []\n\n# Find max values in alternative event dict\nalt_max, alt_values = find_max_values_in_event_dict(alternative_event_dict, "Alternative Event Dict")\n\n# Find max values in or

last event happens at 2.5 min? can't be right. data later on shows this probably isn't the case but check back

In [24]:
3117861 * (1000 / 20000) # converting largest timestamp to ms to see where the last event happened in ms 

155893.05000000002

### Dropping recs in sp.event_dict that aren't in alt event dict

### new mapped code | Dropping recs in sp.event_dict that aren't in alt event dict

In [25]:
def run_zscore_global_baseline_mapped(recording, event_name, alt_event_dict, recording_mapping, pre_window=10, SD=1.65, verbose=False):
    """
    Z-score event firing rates using a *pooled* baseline (all event types) per unit.
    This function calculates the z-score of firing rates for a specific event type
    based on a global baseline computed from all event types in the recording.
    
    MODIFIED: Uses recording mapping to match spike collection names to alt event dict keys
    
    Parameters:
    - recording: SpikeRecording object containing spike data and events.
    - event_name: Name of the event type to analyze.
    - alt_event_dict: Alternative event dictionary to use instead of recording.event_dict
    - recording_mapping: Dictionary mapping spike collection names to alt event dict keys
    - pre_window: Duration in seconds before the event to use for baseline calculation.
    - SD: Number of standard deviations to use for significance thresholding.
    - verbose: If True, prints additional information during processing.
    Returns:
    - A pandas DataFrame containing the z-scores and significance of firing rates for each unit
    for the specified event type.
    """
    # Check if recording has a mapping to alt_event_dict
    alt_recording_name = recording_mapping.get(recording.name)
    if alt_recording_name is None or alt_recording_name not in alt_event_dict:
        if verbose:
            print(f"Recording {recording.name} -> {alt_recording_name} not found in alternative event dict. Skipping.")
        return pd.DataFrame()
    
    # Use the alternative event dict for this recording
    recording_events = alt_event_dict[alt_recording_name]
    
    # Step 1: Pool all baseline windows across all events for each unit
    global_baseline_counts = {}
    units = getattr(recording, "good_units", None) # get good units if available
    if units is None: # if not, use labels_dict
        units = [unit_id for unit_id, label in recording.labels_dict.items() if label == "good"]

        if verbose:
            print("Using labels_dict to determine good units.")
            print(f"Good units found: {units}\n\nFrom labels_dict: {recording.labels_dict}\n\n")

    # Initialize global baseline list per unit
    for unit_id in units:
        global_baseline_counts[unit_id] = []
    if verbose:
        print(f"Global baseline counts initialized for units: {global_baseline_counts}\n")    

    # Loop through all event types and pool all baselines
    # creates a list of baseline counts for each unit
    for ev_type, event_timestamps in recording_events.items():
        # Convert single timestamps to windows if needed
        if len(event_timestamps) > 0 and not isinstance(event_timestamps[0], (list, tuple)):
            # Assume these are single timestamps, create 1-second windows
            event_windows = [[ts, ts + 1000] for ts in event_timestamps]
        else:
            event_windows = event_timestamps
            
        for unit_id in units:
            spikes = recording.unit_timestamps[unit_id] # array of spike timestamps
            spikes_ms = spikes * (1000 / recording.sampling_rate) # convert to milliseconds since event_windows are in ms

            for window in event_windows:
                if len(window) >= 2:
                    start_event = window[0]
                    start_baseline = start_event - int(pre_window * 1000)
                    end_baseline = start_event
                    baseline_count = np.sum((spikes_ms >= start_baseline) & (spikes_ms < end_baseline))
                    global_baseline_counts[unit_id].append(baseline_count) # list of counts for this unit appended to global_baseline_counts

                    if verbose:
                        print(f"Unit {unit_id}, Event {ev_type}, Baseline count: {baseline_count}")

    # Step 2: Compute global baseline mean and SD per unit using numpy
    baseline_mean = {u: np.mean(c) for u, c in global_baseline_counts.items()}
    baseline_sd = {u: np.std(c) for u, c in global_baseline_counts.items()}

    # Step 3: For the target event, calculate z-scores
    if event_name not in recording_events:
        if verbose:
            print(f"Event {event_name} not found in recording {alt_recording_name}")
        return pd.DataFrame()
        
    event_timestamps = recording_events[event_name]
    
    # Convert single timestamps to windows if needed
    if len(event_timestamps) > 0 and not isinstance(event_timestamps[0], (list, tuple)):
        # Assume these are single timestamps, create 1-second windows
        event_windows = [[ts, ts + 1000] for ts in event_timestamps]
    else:
        event_windows = event_timestamps
    
    event_firing = {}
    rows = []
    for unit_id in units:
        spikes = recording.unit_timestamps[unit_id]
        spikes_ms = spikes * (1000 / recording.sampling_rate)
        event_counts = []
        for window in event_windows:
            if len(window) >= 2:
                start_event = window[0]
                end_event = window[1]
                event_count = np.sum((spikes_ms >= start_event) & (spikes_ms < end_event)) # count spikes in the event window using masking
                event_counts.append(event_count)

        # getting all the important values for z-score calculation per unit
        ev_mean = np.mean(event_counts) if event_counts else 0
        b_mean = baseline_mean[unit_id]
        b_sd = baseline_sd[unit_id]

        # Calculate z-score
        zscore = np.nan if b_sd == 0 else (ev_mean - b_mean) / b_sd 

        rows.append({
            "Recording": recording.name,  # Keep original spike collection name
            "Alt_Recording": alt_recording_name,  # Add mapped name for reference
            "Event name": event_name,
            "Unit number": unit_id,
            "Global Pre-event M": b_mean,
            "Global Pre-event SD": b_sd,
            "Event M": ev_mean,
            "Event Z-Score": zscore,
        })

    df = pd.DataFrame(rows)

    # Step 4: Apply significance threshold
    conditions = [
        (df['Event Z-Score'] > SD),
        (df['Event Z-Score'] < -SD)]

    values = ['increase', 'decrease']

    # Apply the conditions
    df['sig'] = np.select(conditions, values, default='not sig')

    return df


def run_all_recordings_all_events_mapped(sp, alt_event_dict, recording_mapping, pre_window=10, SD=1.65, verbose=False):
    """
    Modified version that uses recording mapping to match spike collection names to alt event dict keys
    """
    # Get event names from alt_event_dict instead of sp.recordings
    all_event_names = set()
    for recording_name, recording_events in alt_event_dict.items():
        all_event_names.update(recording_events.keys())
    all_event_names = sorted(all_event_names)
    
    dfs = []
    skipped = []

    for rec in sp.recordings:
        # Check if recording has a valid mapping
        alt_recording_name = recording_mapping.get(rec.name)
        if alt_recording_name is None or alt_recording_name not in alt_event_dict:
            skipped.append((rec.name, "ALL_EVENTS", "no mapping to alt_event_dict"))
            if verbose:
                print(f"Skipping recording {rec.name} - no valid mapping to alternative event dict")
            continue
            
        recording_events = alt_event_dict[alt_recording_name]
        
        for ev in all_event_names:
            if ev not in recording_events or len(recording_events[ev]) == 0:
                skipped.append((rec.name, ev, "no events in alt_event_dict"))
                continue
            try:
                df_ev = run_zscore_global_baseline_mapped(rec, ev, alt_event_dict, recording_mapping, pre_window=pre_window, SD=SD, verbose=verbose)
                if not df_ev.empty:
                    # Extra metadata, if you want it
                    df_ev["Event windows"] = len(recording_events[ev])
                    dfs.append(df_ev)
                else:
                    skipped.append((rec.name, ev, "empty result"))
            except Exception as e:
                skipped.append((rec.name, ev, f"error: {e}"))
                if verbose:
                    print(f"[WARN] {rec.name} / {ev} -> {e}")

    big_df = pd.concat(dfs, ignore_index=True) if dfs else pd.DataFrame()
    return big_df, pd.DataFrame(skipped, columns=["Recording", "Event name", "reason"])

### new testing of mapped funciton |  mapping merged.recs in sp to recs in alt event dict that doesn't include the extra info

In [26]:
# Create a mapping between spike collection recording names and alt event dict keys
def create_recording_name_mapping(sp_recordings, alt_event_dict_keys):
    """
    Create a mapping between spike collection recording names and alternative event dict keys
    """
    mapping = {}
    
    for sp_rec in sp_recordings:
        sp_name = sp_rec.name
        
        # Try to find a match in alt_event_dict_keys
        best_match = None
        for alt_key in alt_event_dict_keys:
            # Check if the alt_key is a substring of the sp_name
            if alt_key in sp_name:
                best_match = alt_key
                break
        
        if best_match:
            mapping[sp_name] = best_match
        else:
            mapping[sp_name] = None
    
    return mapping

# Create the mapping
recording_mapping = create_recording_name_mapping(sp.recordings, alternative_event_dict.keys())

print("Recording name mapping created successfully!")
print(f"Total mappings: {len(recording_mapping)}")
matched_count = sum(1 for v in recording_mapping.values() if v is not None)
print(f"Successfully matched: {matched_count}")
print(f"No matches found: {len(recording_mapping) - matched_count}")

Recording name mapping created successfully!
Total mappings: 39
Successfully matched: 29
No matches found: 10


In [27]:
# Test the mapped functions
print("Testing the new mapping-based functions:")

# Show which recordings will be processed with the mapping
matched_recordings = [rec.name for rec in sp.recordings if recording_mapping.get(rec.name) is not None]
unmatched_recordings = [rec.name for rec in sp.recordings if recording_mapping.get(rec.name) is None]

print(f"\nRecordings that will be processed (with mapping):")
for rec_name in matched_recordings:
    alt_name = recording_mapping[rec_name]
    print(f"  {rec_name} -> {alt_name}")

print(f"\nRecordings that will still be skipped:")
for rec_name in unmatched_recordings:
    print(f"  {rec_name} -> NO MATCH")

print(f"\nProcessing summary:")
print(f"Total SP recordings: {len(sp.recordings)}")
print(f"Will be processed: {len(matched_recordings)}")
print(f"Will be skipped: {len(unmatched_recordings)}")

# Test with a single recording to make sure it works
if matched_recordings:
    test_rec_name = matched_recordings[0]
    test_rec = next(rec for rec in sp.recordings if rec.name == test_rec_name)
    alt_rec_name = recording_mapping[test_rec_name]
    
    print(f"\nTesting with first matched recording:")
    print(f"SP recording: {test_rec_name}")
    print(f"Alt recording: {alt_rec_name}")
    print(f"Events in alt recording: {list(alternative_event_dict[alt_rec_name].keys())}")
    
    # Test processing one event
    first_event = list(alternative_event_dict[alt_rec_name].keys())[0]
    print(f"Testing event: {first_event}")
    print(f"Number of timestamps for this event: {len(alternative_event_dict[alt_rec_name][first_event])}")
else:
    print("\nNo matched recordings found to test with!")

Testing the new mapping-based functions:

Recordings that will be processed (with mapping):
  20230612_101430_standard_comp_to_training_D1_subj_1-3_t3b3L_box2_merged.rec -> 20230612_101430_standard_comp_to_training_D1_subj_1-3
  20230612_101430_standard_comp_to_training_D1_subj_1-4_t4b2L_box1_merged.rec -> 20230612_101430_standard_comp_to_training_D1_subj_1-4
  20230612_112630_standard_comp_to_training_D1_subj_1-1_t1b3L_box2_merged.rec -> 20230612_112630_standard_comp_to_training_D1_subj_1-1
  20230612_112630_standard_comp_to_training_D1_subj_1-2_t2b2L_box1_merged.rec -> 20230612_112630_standard_comp_to_training_D1_subj_1-2
  20230613_105657_standard_comp_to_training_D2_subj_1-1_t1b2L_box1_merged.rec -> 20230613_105657_standard_comp_to_training_D2_subj_1-1
  20230613_105657_standard_comp_to_training_D2_subj_1-4_t4b3L_box2_merged.rec -> 20230613_105657_standard_comp_to_training_D2_subj_1-4
  20230614_114041_standard_comp_to_training_D3_subj_1-1_t1b3L_box1_merged.rec -> 20230614_114041_s

In [28]:
# Run the analysis using the alternative event dictionary
print("Running z-score analysis with alternative event dictionary...")

# Run the analysis
big_df, skipped_df = run_all_recordings_all_events_mapped(
    sp, 
    alternative_event_dict, 
    recording_mapping, 
    pre_window=10, 
    SD=1.65, 
    verbose=False  # Set to True if you want detailed output
)

print(f"\nAnalysis complete!")
print(f"Results shape: {big_df.shape}")
print(f"Skipped entries: {skipped_df.shape[0]}")

if not big_df.empty:
    print(f"\nFirst few rows of results:")
    print(big_df.head())
    
    print(f"\nUnique recordings processed: {big_df['Recording'].nunique()}")
    print(f"Unique events analyzed: {big_df['Event name'].nunique()}")
    print(f"Event types: {sorted(big_df['Event name'].unique())}")
else:
    print("No results generated!")

if not skipped_df.empty:
    print(f"\nSkipped entries breakdown:")
    print(skipped_df['reason'].value_counts())

Running z-score analysis with alternative event dictionary...

Analysis complete!
Results shape: (0, 0)
Skipped entries: 242
No results generated!

Skipped entries breakdown:
reason
no events in alt_event_dict                                                   123
no mapping to alt_event_dict                                                   10
error: operands could not be broadcast together with shapes (22282,) (2,)       5
error: operands could not be broadcast together with shapes (25021,) (2,)       5
error: operands could not be broadcast together with shapes (6226,) (2,)        5
error: operands could not be broadcast together with shapes (13468,) (2,)       5
error: operands could not be broadcast together with shapes (23002,) (2,)       5
error: operands could not be broadcast together with shapes (5318,) (2,)        4
error: operands could not be broadcast together with shapes (1174,) (2,)        4
error: operands could not be broadcast together with shapes (395,) (2,)         

In [29]:
# Save the results if you want to
if not big_df.empty:
    # Uncomment the lines below to save the results
    # output_path = r'C:\Users\thoma\Code\ResearchCode\diff_fam_social_memory_ephys\other_peoples_sutff\Thomas\reward_comp\testing_eventdict_creation\zscore_results_alternative_events.csv'
    # big_df.to_csv(output_path, index=False)
    # print(f"Results saved to: {output_path}")
    
    # Show some summary statistics
    print("Summary statistics:")
    print(f"Total units analyzed: {big_df['Unit number'].nunique()}")
    print(f"Total recordings: {big_df['Recording'].nunique()}")
    print(f"Total events: {big_df['Event name'].nunique()}")
    
    # Show significance distribution
    print(f"\nSignificance distribution:")
    print(big_df['sig'].value_counts())
    
    # Show z-score statistics
    print(f"\nZ-score statistics:")
    print(big_df['Event Z-Score'].describe())
else:
    print("No results to save.")

No results to save.


### Simple Direct Approach - Use Alt Event Dict Directly

In [ ]:
def run_zscore_direct_alt_events_fixed(sp, alt_event_dict, pre_window=10, SD=1.65, verbose=False):
    """
    Run z-score analysis using ONLY the alternative event dictionary.
    No mapping needed - directly uses alt_event_dict structure.
    
    Parameters:
    - sp: SpikeCollection object (for spike data only)
    - alt_event_dict: Alternative event dictionary with structure {recording_name: {event_name: [timestamps]}}
    - pre_window: Duration in seconds before event for baseline
    - SD: Standard deviation threshold for significance
    - verbose: Print debug info
    """
    
    all_results = []
    skipped = []
    
    # Get all unique event names from alt_event_dict
    all_event_names = set()
    for rec_events in alt_event_dict.values():
        all_event_names.update(rec_events.keys())
    all_event_names = sorted(all_event_names)
    
    if verbose:
        print(f"Events found in alt_event_dict: {all_event_names}")
    
    # Process each recording in alt_event_dict
    for alt_rec_name, events_dict in alt_event_dict.items():
        if verbose:
            print(f"\nProcessing recording: {alt_rec_name}")
        
        # Find matching spike collection recording (by substring matching)
        spike_recording = None
        for sp_rec in sp.recordings:
            if alt_rec_name in sp_rec.name:  # Simple substring match
                spike_recording = sp_rec
                break
        
        if spike_recording is None:
            skipped.append((alt_rec_name, "ALL_EVENTS", "no matching spike recording"))
            if verbose:
                print(f"  No matching spike recording found for {alt_rec_name}")
            continue
        
        if verbose:
            print(f"  Matched to spike recording: {spike_recording.name}")
        
        # Get good units
        units = getattr(spike_recording, "good_units", None)
        if units is None:
            units = [unit_id for unit_id, label in spike_recording.labels_dict.items() if label == "good"]
        
        if not units:
            skipped.append((alt_rec_name, "ALL_EVENTS", "no good units"))
            continue
        
        # Step 1: Pool ALL baselines across ALL events for each unit
        global_baseline_counts = {unit_id: [] for unit_id in units}
        
        for event_name, event_windows in events_dict.items():
            if len(event_windows) == 0:
                continue
                
            for unit_id in units:
                spikes = spike_recording.unit_timestamps[unit_id]
                spikes_ms = spikes * (1000 / spike_recording.sampling_rate)
                
                # For each event window, calculate baseline before the start
                for window in event_windows:
                    if len(window) >= 2:  # Should be [start, end]
                        event_start = window[0]
                        start_baseline = event_start - int(pre_window * 1000)
                        end_baseline = event_start
                        baseline_count = np.sum((spikes_ms >= start_baseline) & (spikes_ms < end_baseline))
                        global_baseline_counts[unit_id].append(baseline_count)
        
        # Step 2: Calculate global baseline stats per unit
        baseline_mean = {u: np.mean(counts) if counts else 0 for u, counts in global_baseline_counts.items()}
        baseline_sd = {u: np.std(counts) if counts else 0 for u, counts in global_baseline_counts.items()}
        
        # Step 3: Calculate z-scores for each event
        for event_name in all_event_names:
            if event_name not in events_dict or len(events_dict[event_name]) == 0:
                skipped.append((alt_rec_name, event_name, "no events"))
                continue
            
            event_timestamps = events_dict[event_name]
            
            for unit_id in units:
                spikes = spike_recording.unit_timestamps[unit_id]
                spikes_ms = spikes * (1000 / spike_recording.sampling_rate)
                
                # Count spikes for each event occurrence (1 second windows)
                event_counts = []
                for timestamp in event_timestamps:
                    start_event = timestamp
                    end_event = timestamp + 1000  # 1 second window
                    event_count = np.sum((spikes_ms >= start_event) & (spikes_ms < end_event))
                    event_counts.append(event_count)
                
                # Calculate stats
                ev_mean = np.mean(event_counts) if event_counts else 0
                b_mean = baseline_mean[unit_id]
                b_sd = baseline_sd[unit_id]
                
                # Z-score calculation
                zscore = np.nan if b_sd == 0 else (ev_mean - b_mean) / b_sd
                
                # Significance
                if np.isnan(zscore):
                    sig = "not sig"
                elif zscore > SD:
                    sig = "increase"
                elif zscore < -SD:
                    sig = "decrease"
                else:
                    sig = "not sig"
                
                all_results.append({
                    "Alt_Recording": alt_rec_name,
                    "SP_Recording": spike_recording.name,
                    "Event name": event_name,
                    "Unit number": unit_id,
                    "Global Pre-event M": b_mean,
                    "Global Pre-event SD": b_sd,
                    "Event M": ev_mean,
                    "Event Z-Score": zscore,
                    "sig": sig,
                    "Event count": len(event_timestamps)
                })
    
    # Convert to DataFrame
    results_df = pd.DataFrame(all_results) if all_results else pd.DataFrame()
    skipped_df = pd.DataFrame(skipped, columns=["Recording", "Event name", "reason"]) if skipped else pd.DataFrame()
    
    return results_df, skipped_df

In [32]:
# Test the direct approach - no mapping needed!
print("Running z-score analysis directly with alternative event dictionary...")
print("This bypasses all spike collection events and uses ONLY your alt_event_dict")

# Run the simple direct analysis
results_df, skipped_df = run_zscore_direct_alt_events(
    sp, 
    alternative_event_dict, 
    pre_window=10, 
    SD=1.65, 
    verbose=True  # Set to False to reduce output
)

print(f"\n{'='*50}")
print(f"DIRECT ANALYSIS COMPLETE!")
print(f"{'='*50}")
print(f"Results shape: {results_df.shape}")
print(f"Skipped entries: {skipped_df.shape[0]}")

if not results_df.empty:
    print(f"\nFirst few rows:")
    print(results_df.head())
    
    print(f"\nSummary:")
    print(f"- Recordings processed: {results_df['Alt_Recording'].nunique()}")
    print(f"- Event types: {results_df['Event name'].nunique()}")
    print(f"- Total units: {results_df['Unit number'].nunique()}")
    print(f"- Event types found: {sorted(results_df['Event name'].unique())}")
    
    print(f"\nSignificance counts:")
    print(results_df['sig'].value_counts())
    
else:
    print("❌ No results generated!")

if not skipped_df.empty:
    print(f"\nSkipped breakdown:")
    print(skipped_df['reason'].value_counts())

Running z-score analysis directly with alternative event dictionary...
This bypasses all spike collection events and uses ONLY your alt_event_dict
Events found in alt_event_dict: ['both_rewarded', 'competitive_lose', 'competitive_win', 'no_comp_lose', 'no_comp_win', 'omission', 'rewarded', 'tie']

Processing recording: 20230612_101430_standard_comp_to_training_D1_subj_1-3
  Matched to spike recording: 20230612_101430_standard_comp_to_training_D1_subj_1-3_t3b3L_box2_merged.rec


ValueError: operands could not be broadcast together with shapes (395,) (2,) 

In [34]:
# First, let's inspect the structure of your alternative event dictionary
test_rec_name = "20230612_101430_standard_comp_to_training_D1_subj_1-3"
if test_rec_name in alternative_event_dict:
    events = alternative_event_dict[test_rec_name]
    print("Event structure analysis:")
    for event_name, event_data in events.items():
        print(f"\nEvent: {event_name}")
        print(f"  Type: {type(event_data)}")
        print(f"  Shape: {np.array(event_data).shape}")
        if len(event_data) > 0:
            print(f"  First few values: {event_data[:3]}")
            print(f"  Value types: {type(event_data[0]) if len(event_data) > 0 else 'N/A'}")
            break  # Found a non-empty event, stop here
    
    # Find a non-empty event
    print("\nLooking for non-empty events:")
    for event_name, event_data in events.items():
        if len(event_data) > 0:
            print(f"Non-empty event found: {event_name}")
            print(f"  Shape: {np.array(event_data).shape}")
            print(f"  First value: {event_data[0]}")
            print(f"  Type of first value: {type(event_data[0])}")
            break

Event structure analysis:

Event: rewarded
  Type: <class 'numpy.ndarray'>
  Shape: (0,)

Event: omission
  Type: <class 'numpy.ndarray'>
  Shape: (0,)

Event: both_rewarded
  Type: <class 'numpy.ndarray'>
  Shape: (0,)

Event: tie
  Type: <class 'numpy.ndarray'>
  Shape: (0,)

Event: no_comp_win
  Type: <class 'numpy.ndarray'>
  Shape: (5, 2)
  First few values: [[169111 179111]
 [479111 489111]
 [804110 814110]]
  Value types: <class 'numpy.ndarray'>

Looking for non-empty events:
Non-empty event found: no_comp_win
  Shape: (5, 2)
  First value: [169111 179111]
  Type of first value: <class 'numpy.ndarray'>


In [36]:
def run_zscore_direct_alt_events_fixed(sp, alt_event_dict, pre_window=10, SD=1.65, verbose=False):
    """
    Run z-score analysis using ONLY the alternative event dictionary.
    FIXED: Handles event windows [start, end] instead of single timestamps.
    
    Parameters:
    - sp: SpikeCollection object (for spike data only)
    - alt_event_dict: Alternative event dictionary with structure {recording_name: {event_name: [[start, end], ...]}}
    - pre_window: Duration in seconds before event for baseline
    - SD: Standard deviation threshold for significance
    - verbose: Print debug info
    """
    
    all_results = []
    skipped = []
    
    # Get all unique event names from alt_event_dict
    all_event_names = set()
    for rec_events in alt_event_dict.values():
        all_event_names.update(rec_events.keys())
    all_event_names = sorted(all_event_names)
    
    if verbose:
        print(f"Events found in alt_event_dict: {all_event_names}")
    
    # Process each recording in alt_event_dict
    for alt_rec_name, events_dict in alt_event_dict.items():
        if verbose:
            print(f"\nProcessing recording: {alt_rec_name}")
        
        # Find matching spike collection recording (by substring matching)
        spike_recording = None
        for sp_rec in sp.recordings:
            if alt_rec_name in sp_rec.name:  # Simple substring match
                spike_recording = sp_rec
                break
        
        if spike_recording is None:
            skipped.append((alt_rec_name, "ALL_EVENTS", "no matching spike recording"))
            if verbose:
                print(f"  No matching spike recording found for {alt_rec_name}")
            continue
        
        if verbose:
            print(f"  Matched to spike recording: {spike_recording.name}")
        
        # Get good units
        units = getattr(spike_recording, "good_units", None)
        if units is None:
            units = [unit_id for unit_id, label in spike_recording.labels_dict.items() if label == "good"]
        
        if not units:
            skipped.append((alt_rec_name, "ALL_EVENTS", "no good units"))
            continue
        
        # Step 1: Pool ALL baselines across ALL events for each unit
        global_baseline_counts = {unit_id: [] for unit_id in units}
        
        for event_name, event_windows in events_dict.items():
            if len(event_windows) == 0:
                continue
                
            for unit_id in units:
                spikes = spike_recording.unit_timestamps[unit_id]
                spikes_ms = spikes * (1000 / spike_recording.sampling_rate)
                
                # For each event window, calculate baseline before the start
                for window in event_windows:
                    if len(window) >= 2:  # Should be [start, end]
                        event_start = window[0]
                        start_baseline = event_start - int(pre_window * 1000)
                        end_baseline = event_start
                        baseline_count = np.sum((spikes_ms >= start_baseline) & (spikes_ms < end_baseline))
                        global_baseline_counts[unit_id].append(baseline_count)
        
        # Step 2: Calculate global baseline stats per unit
        baseline_mean = {u: np.mean(counts) if counts else 0 for u, counts in global_baseline_counts.items()}
        baseline_sd = {u: np.std(counts) if counts else 0 for u, counts in global_baseline_counts.items()}
        
        # Step 3: Calculate z-scores for each event
        for event_name in all_event_names:
            if event_name not in events_dict or len(events_dict[event_name]) == 0:
                skipped.append((alt_rec_name, event_name, "no events"))
                continue
            
            event_windows = events_dict[event_name]
            
            for unit_id in units:
                spikes = spike_recording.unit_timestamps[unit_id]
                spikes_ms = spikes * (1000 / spike_recording.sampling_rate)
                
                # Count spikes for each event window
                event_counts = []
                for window in event_windows:
                    if len(window) >= 2:
                        start_event = window[0]
                        end_event = window[1]
                        event_count = np.sum((spikes_ms >= start_event) & (spikes_ms < end_event))
                        event_counts.append(event_count)
                
                # Calculate stats
                ev_mean = np.mean(event_counts) if event_counts else 0
                b_mean = baseline_mean[unit_id]
                b_sd = baseline_sd[unit_id]
                
                # Z-score calculation
                zscore = np.nan if b_sd == 0 else (ev_mean - b_mean) / b_sd
                
                # Significance
                if np.isnan(zscore):
                    sig = "not sig"
                elif zscore > SD:
                    sig = "increase"
                elif zscore < -SD:
                    sig = "decrease"
                else:
                    sig = "not sig"
                
                all_results.append({
                    "Alt_Recording": alt_rec_name,
                    "SP_Recording": spike_recording.name,
                    "Event name": event_name,
                    "Unit number": unit_id,
                    "Global Pre-event M": b_mean,
                    "Global Pre-event SD": b_sd,
                    "Event M": ev_mean,
                    "Event Z-Score": zscore,
                    "sig": sig,
                    "Event count": len(event_windows)
                })
    
    # Convert to DataFrame
    results_df = pd.DataFrame(all_results) if all_results else pd.DataFrame()
    skipped_df = pd.DataFrame(skipped, columns=["Recording", "Event name", "reason"]) if skipped else pd.DataFrame()
    
    return results_df, skipped_df

In [37]:
# Test the FIXED direct approach
print("🚀 Running FIXED z-score analysis with alternative event dictionary...")
print("Now correctly handles [start, end] time windows!")

# Run the corrected analysis
results_df, skipped_df = run_zscore_direct_alt_events_fixed(
    sp, 
    alternative_event_dict, 
    pre_window=10, 
    SD=1.65, 
    verbose=False  # Change to True for detailed output
)

print(f"\n{'='*50}")
print(f"✅ ANALYSIS COMPLETE!")
print(f"{'='*50}")
print(f"Results shape: {results_df.shape}")
print(f"Skipped entries: {skipped_df.shape[0]}")

if not results_df.empty:
    print(f"\n📊 SUCCESS! Generated {results_df.shape[0]} results")
    print(f"\nFirst few rows:")
    print(results_df.head())
    
    print(f"\n📈 Summary:")
    print(f"- Recordings processed: {results_df['Alt_Recording'].nunique()}")
    print(f"- Event types: {results_df['Event name'].nunique()}")
    print(f"- Total units: {results_df['Unit number'].nunique()}")
    print(f"- Event types found: {sorted(results_df['Event name'].unique())}")
    
    print(f"\n🎯 Significance counts:")
    sig_counts = results_df['sig'].value_counts()
    for sig_type, count in sig_counts.items():
        print(f"  {sig_type}: {count}")
    
    print(f"\n📊 Z-score statistics:")
    print(results_df['Event Z-Score'].describe())
    
else:
    print("❌ No results generated!")

if not skipped_df.empty:
    print(f"\n⚠️  Skipped breakdown:")
    print(skipped_df['reason'].value_counts())

🚀 Running FIXED z-score analysis with alternative event dictionary...
Now correctly handles [start, end] time windows!

✅ ANALYSIS COMPLETE!
Results shape: (1908, 10)
Skipped entries: 134

📊 SUCCESS! Generated 1908 results

First few rows:
                                           Alt_Recording  \
0  20230612_101430_standard_comp_to_training_D1_subj_1-3   
1  20230612_101430_standard_comp_to_training_D1_subj_1-3   
2  20230612_101430_standard_comp_to_training_D1_subj_1-3   
3  20230612_101430_standard_comp_to_training_D1_subj_1-3   
4  20230612_101430_standard_comp_to_training_D1_subj_1-3   

                                                                  SP_Recording  \
0  20230612_101430_standard_comp_to_training_D1_subj_1-3_t3b3L_box2_merged.rec   
1  20230612_101430_standard_comp_to_training_D1_subj_1-3_t3b3L_box2_merged.rec   
2  20230612_101430_standard_comp_to_training_D1_subj_1-3_t3b3L_box2_merged.rec   
3  20230612_101430_standard_comp_to_training_D1_subj_1-3_t3b3L_box2_mer

In [38]:
results_df.head()

,Alt_Recording,SP_Recording,Event name,Unit number,Global Pre-event M,Global Pre-event SD,Event M,Event Z-Score,sig,Event count
0,20230612_101430_standard_comp_to_training_D1_subj_1-3,20230612_101430_standard_comp_to_training_D1_subj_1-3_t3b3L_box2_merged.rec,competitive_lose,104,1.30,2.776689,1.0,-0.108042,not sig,1
1,20230612_101430_standard_comp_to_training_D1_subj_1-3,20230612_101430_standard_comp_to_training_D1_subj_1-3_t3b3L_box2_merged.rec,competitive_lose,11,8.10,8.086408,0.0,-1.001681,not sig,1
2,20230612_101430_standard_comp_to_training_D1_subj_1-3,20230612_101430_standard_comp_to_training_D1_subj_1-3_t3b3L_box2_merged.rec,competitive_lose,122,0.40,0.583095,0.0,-0.685994,not sig,1
3,20230612_101430_standard_comp_to_training_D1_subj_1-3,20230612_101430_standard_comp_to_training_D1_subj_1-3_t3b3L_box2_merged.rec,competitive_lose,125,6.35,4.077683,4.0,-0.576308,not sig,1
4,20230612_101430_standard_comp_to_training_D1_subj_1-3,20230612_101430_standard_comp_to_training_D1_subj_1-3_t3b3L_box2_merged.rec,competitive_lose,126,16.50,25.482347,4.0,-0.490536,not sig,1


In [43]:
results_df['Event name'].unique()

array(['competitive_lose', 'competitive_win', 'no_comp_lose',
       'no_comp_win', 'tie'], dtype=object)

In [39]:
skipped_df.head()

,Recording,Event name,reason
0,20230612_101430_standard_comp_to_training_D1_subj_1-3,both_rewarded,no events
1,20230612_101430_standard_comp_to_training_D1_subj_1-3,omission,no events
2,20230612_101430_standard_comp_to_training_D1_subj_1-3,rewarded,no events
3,20230612_101430_standard_comp_to_training_D1_subj_1-3,tie,no events
4,20230612_101430_standard_comp_to_training_D1_subj_1-4,both_rewarded,no events


### Event Name Mapping for Analysis Compatibility

In [40]:
# Create mapping from your analysis event names to alt_event_dict names
def create_event_name_mapping():
    """
    Map analysis event names to alternative event dictionary names
    """
    # Your analysis events: ['alone_rewarded', 'alone_rewarded_baseline', 'high_comp', 'high_comp_lose', 
    # 'high_comp_lose_baseline', 'high_comp_win', 'high_comp_win_baseline', 'lose', 'low_comp', 
    # 'low_comp_lose', 'low_comp_lose_baseline', 'low_comp_win', 'low_comp_win_baseline', 'overall_pretone', 'win']
    
    # Alt event dict events: ['both_rewarded', 'competitive_lose', 'competitive_win', 'no_comp_lose', 'no_comp_win', 'omission', 'rewarded', 'tie']
    
    event_mapping = {
        # Map no_comp events to low_comp equivalents
        'no_comp_win': 'low_comp_win',
        'no_comp_lose': 'low_comp_lose', 
        
        # Map competitive events to high_comp equivalents  
        'competitive_win': 'high_comp_win',
        'competitive_lose': 'high_comp_lose',
        
        # Map rewarded events
        'rewarded': 'alone_rewarded',
        'both_rewarded': 'win',  # or could be 'high_comp' depending on your preference
        
        # Events that don't have clear mappings - keep as is or skip
        'omission': None,  # Skip this one
        'tie': None,       # Skip this one
    }
    
    return event_mapping

# Create the mapping
event_mapping = create_event_name_mapping()

print("Event name mapping:")
for alt_name, analysis_name in event_mapping.items():
    if analysis_name:
        print(f"  {alt_name} -> {analysis_name}")
    else:
        print(f"  {alt_name} -> SKIP")

# Show what events are available in alt_event_dict
print(f"\nEvents in alt_event_dict:")
alt_events = set()
for rec_events in alternative_event_dict.values():
    alt_events.update(rec_events.keys())
print(f"  {sorted(alt_events)}")

print(f"\nYour analysis expects these events:")
analysis_events = ['alone_rewarded', 'alone_rewarded_baseline', 'high_comp', 'high_comp_lose', 
                  'high_comp_lose_baseline', 'high_comp_win', 'high_comp_win_baseline', 'lose', 'low_comp', 
                  'low_comp_lose', 'low_comp_lose_baseline', 'low_comp_win', 'low_comp_win_baseline', 'overall_pretone', 'win']
print(f"  {analysis_events}")

Event name mapping:
  no_comp_win -> low_comp_win
  no_comp_lose -> low_comp_lose
  competitive_win -> high_comp_win
  competitive_lose -> high_comp_lose
  rewarded -> alone_rewarded
  both_rewarded -> win
  omission -> SKIP
  tie -> SKIP

Events in alt_event_dict:
  ['both_rewarded', 'competitive_lose', 'competitive_win', 'no_comp_lose', 'no_comp_win', 'omission', 'rewarded', 'tie']

Your analysis expects these events:
  ['alone_rewarded', 'alone_rewarded_baseline', 'high_comp', 'high_comp_lose', 'high_comp_lose_baseline', 'high_comp_win', 'high_comp_win_baseline', 'lose', 'low_comp', 'low_comp_lose', 'low_comp_lose_baseline', 'low_comp_win', 'low_comp_win_baseline', 'overall_pretone', 'win']


In [41]:
# Apply event name mapping to results DataFrame
def apply_event_mapping_to_results(df, event_mapping):
    """
    Apply event name mapping to the results DataFrame
    """
    if df.empty:
        print("DataFrame is empty, nothing to map")
        return df
    
    print(f"Original DataFrame shape: {df.shape}")
    print(f"Original events: {sorted(df['Event name'].unique())}")
    
    # Create a copy to avoid modifying original
    mapped_df = df.copy()
    
    # Apply the mapping
    mapped_df['Event name'] = mapped_df['Event name'].map(event_mapping).fillna(mapped_df['Event name'])
    
    # Remove rows where events should be skipped (mapped to None)
    original_size = len(mapped_df)
    mapped_df = mapped_df[mapped_df['Event name'].notna()]
    skipped_count = original_size - len(mapped_df)
    
    print(f"Mapped DataFrame shape: {mapped_df.shape}")
    print(f"Mapped events: {sorted(mapped_df['Event name'].unique())}")
    print(f"Rows skipped (unmapped events): {skipped_count}")
    
    return mapped_df

# Apply the mapping to your results
print("Applying event name mapping to results...")
mapped_results_df = apply_event_mapping_to_results(results_df, event_mapping)

print(f"\n📊 Mapping Summary:")
print(f"- Original results: {results_df.shape[0]} rows")
print(f"- Mapped results: {mapped_results_df.shape[0]} rows") 
print(f"- Events after mapping: {sorted(mapped_results_df['Event name'].unique())}")

# Show a sample of the mapped results
if not mapped_results_df.empty:
    print(f"\nSample of mapped results:")
    print(mapped_results_df[['Alt_Recording', 'Event name', 'Unit number', 'Event Z-Score', 'sig']].head(10))

Applying event name mapping to results...
Original DataFrame shape: (1908, 10)
Original events: ['competitive_lose', 'competitive_win', 'no_comp_lose', 'no_comp_win', 'tie']
Mapped DataFrame shape: (1908, 10)
Mapped events: ['high_comp_lose', 'high_comp_win', 'low_comp_lose', 'low_comp_win', 'tie']
Rows skipped (unmapped events): 0

📊 Mapping Summary:
- Original results: 1908 rows
- Mapped results: 1908 rows
- Events after mapping: ['high_comp_lose', 'high_comp_win', 'low_comp_lose', 'low_comp_win', 'tie']

Sample of mapped results:
                                           Alt_Recording      Event name  \
0  20230612_101430_standard_comp_to_training_D1_subj_1-3  high_comp_lose   
1  20230612_101430_standard_comp_to_training_D1_subj_1-3  high_comp_lose   
2  20230612_101430_standard_comp_to_training_D1_subj_1-3  high_comp_lose   
3  20230612_101430_standard_comp_to_training_D1_subj_1-3  high_comp_lose   
4  20230612_101430_standard_comp_to_training_D1_subj_1-3  high_comp_lose   
5  20

In [42]:
# Save the mapped results and create analysis-ready data
print("Creating analysis-ready dataset...")

# Remove 'tie' events if you want to skip them for your analysis
analysis_ready_df = mapped_results_df[mapped_results_df['Event name'] != 'tie'].copy()

print(f"\nFinal analysis-ready dataset:")
print(f"- Shape: {analysis_ready_df.shape}")
print(f"- Events included: {sorted(analysis_ready_df['Event name'].unique())}")
print(f"- Recordings: {analysis_ready_df['Alt_Recording'].nunique()}")
print(f"- Units: {analysis_ready_df['Unit number'].nunique()}")

# Show event counts
print(f"\nEvent counts:")
event_counts = analysis_ready_df['Event name'].value_counts()
for event, count in event_counts.items():
    print(f"  {event}: {count}")

# Show significance distribution
print(f"\nSignificance distribution:")
sig_counts = analysis_ready_df['sig'].value_counts()
for sig_type, count in sig_counts.items():
    print(f"  {sig_type}: {count}")

# Optional: Save the mapped results
save_results = False  # Change to True if you want to save
if save_results:
    output_path = r'C:\Users\thoma\Code\ResearchCode\diff_fam_social_memory_ephys\other_peoples_sutff\Thomas\reward_comp\testing_eventdict_creation\mapped_zscore_results.csv'
    analysis_ready_df.to_csv(output_path, index=False)
    print(f"\n💾 Results saved to: {output_path}")

print(f"\n✅ Your mapped dataset is ready!")
print(f"Use 'analysis_ready_df' for your analyses - it contains the correctly mapped event names.")

Creating analysis-ready dataset...

Final analysis-ready dataset:
- Shape: (1808, 10)
- Events included: ['high_comp_lose', 'high_comp_win', 'low_comp_lose', 'low_comp_win']
- Recordings: 29
- Units: 231

Event counts:
  high_comp_lose: 475
  low_comp_win: 458
  low_comp_lose: 449
  high_comp_win: 426

Significance distribution:
  not sig: 1679
  increase: 114
  decrease: 15

✅ Your mapped dataset is ready!
Use 'analysis_ready_df' for your analyses - it contains the correctly mapped event names.


In [44]:
units_df = analysis_ready_df[analysis_ready_df['sig'] != 'not sig']
units_df = units_df.drop(["Global Pre-event M", "Global Pre-event SD", "Event M"], axis=1)

In [45]:
df = units_df.copy()
event_df = df[df['Event name'] == 'high_comp_win']

significant_units = event_df['Unit number'].unique()
print(f"Number of significant units for high_comp_win: {len(significant_units)}")

Number of significant units for high_comp_win: 21


In [46]:
units_df.head(10)

,Alt_Recording,SP_Recording,Event name,Unit number,Event Z-Score,sig,Event count
37,20230612_101430_standard_comp_to_training_D1_subj_1-3,20230612_101430_standard_comp_to_training_D1_subj_1-3_t3b3L_box2_merged.rec,high_comp_win,232,2.061273,increase,8
81,20230612_101430_standard_comp_to_training_D1_subj_1-3,20230612_101430_standard_comp_to_training_D1_subj_1-3_t3b3L_box2_merged.rec,low_comp_win,232,1.749792,increase,5
90,20230612_101430_standard_comp_to_training_D1_subj_1-4,20230612_101430_standard_comp_to_training_D1_subj_1-4_t4b2L_box1_merged.rec,high_comp_lose,46,1.768666,increase,8
97,20230612_101430_standard_comp_to_training_D1_subj_1-4,20230612_101430_standard_comp_to_training_D1_subj_1-4_t4b2L_box1_merged.rec,high_comp_lose,68,1.831168,increase,8
102,20230612_101430_standard_comp_to_training_D1_subj_1-4,20230612_101430_standard_comp_to_training_D1_subj_1-4_t4b2L_box1_merged.rec,high_comp_win,50,2.673358,increase,1
104,20230612_101430_standard_comp_to_training_D1_subj_1-4,20230612_101430_standard_comp_to_training_D1_subj_1-4_t4b2L_box1_merged.rec,high_comp_win,52,2.187709,increase,1
106,20230612_101430_standard_comp_to_training_D1_subj_1-4,20230612_101430_standard_comp_to_training_D1_subj_1-4_t4b2L_box1_merged.rec,high_comp_win,61,1.727083,increase,1
107,20230612_101430_standard_comp_to_training_D1_subj_1-4,20230612_101430_standard_comp_to_training_D1_subj_1-4_t4b2L_box1_merged.rec,high_comp_win,68,1.972027,increase,1
124,20230612_101430_standard_comp_to_training_D1_subj_1-4,20230612_101430_standard_comp_to_training_D1_subj_1-4_t4b2L_box1_merged.rec,low_comp_win,52,1.719472,increase,6
127,20230612_101430_standard_comp_to_training_D1_subj_1-4,20230612_101430_standard_comp_to_training_D1_subj_1-4_t4b2L_box1_merged.rec,low_comp_win,68,10.799193,increase,6


In [47]:
# using os to use functions from this py file C:\Users\thoma\Code\ResearchCode\diff_fam_social_memory_ephys\other_peoples_sutff\Thomas\reward_comp\venn_upset_plot_creation.py
import os
import sys
module_path = r"C:/Users/thoma/Code/ResearchCode/diff_fam_social_memory_ephys/other_peoples_sutff/Thomas/reward_comp"
sys.path.append(module_path)

from venn_upset_plot_creation import create_venn_diagram, analyze_event_overlap, create_overlap_visualization

In [48]:
# Example 1: Compare 'alone_rewarded', 'win', and 'lose' events (your specific example)
print("EXAMPLE 1: Alone Rewarded vs Win vs Lose Events")
events_to_compare = ['alone_rewarded', 'win', 'lose']
_ = create_venn_diagram(units_df, events_to_compare, significance_type='both', 
                        title="All Significant Units");

EXAMPLE 1: Alone Rewarded vs Win vs Lose Events
Significant units for increase: []
Significant units for decrease: []
All significant units: []
alone_rewarded: 0 significant units
Significant units for increase: []
Significant units for decrease: []
All significant units: []
win: 0 significant units
Significant units for increase: []
Significant units for decrease: []
All significant units: []
lose: 0 significant units


KeyError: 'Recording'